# Part 3: Add Tools for Fact-Checking

This tutorial builds on Part 2 by adding search tools to verify claims with real data.

In [22]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.m5_tools import check_claim_with_tools
from config.llm_factory import LLMFactory

from typing import TypedDict, Optional, List
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

print("✅ Tool-enhanced BS detector ready!")

✅ Tool-enhanced BS detector ready!


## Define Tools

Let's create a simple search tool for aviation facts:

In [23]:
@tool
def search_aviation_facts(query: str) -> str:
    """Search for aviation facts to verify claims."""
    # Simple mock database of aviation facts
    facts = {
        "boeing 747 engines": "The Boeing 747 has 4 engines",
        "commercial altitude": "Commercial jets typically fly at 30,000-40,000 feet",
        "speed of sound": "The speed of sound is approximately 767 mph at sea level",
        "wright brothers": "The Wright brothers first flew on December 17, 1903",
        "jet fuel": "Jet fuel is a type of aviation turbine fuel (kerosene-based)"
    }
    
    # Simple search
    query_lower = query.lower()
    for key, fact in facts.items():
        if key in query_lower:
            return fact
    
    return "No specific information found for this query."

# List of tools
tools = [search_aviation_facts]

## Update State for Tools

In [24]:
from langchain_core.messages import BaseMessage

class State(TypedDict):
    claim: str
    messages: List[BaseMessage]
    verdict: Optional[str]
    confidence: Optional[int]
    reasoning: Optional[str]
    search_performed: bool

## Create the BS Detector with Tools

In [25]:
def bs_detector(state: State):
    """BS detector that can use tools"""
    # Create LLM with tools
    llm = LLMFactory.create_llm()
    llm_with_tools = llm.bind_tools(tools)
    
    # Create messages
    from langchain_core.messages import SystemMessage, HumanMessage
    
    # Build message list properly
    if state.get('messages'):
        # If we have existing messages, continue the conversation
        messages = state['messages']
    else:
        # First time - create initial messages
        messages = [
            SystemMessage(content="""You are an aviation expert BS detector. 
            Use the search tool to verify claims when needed.
            Determine if claims are BS (false) or LEGITIMATE (true)."""),
            HumanMessage(content=f"Check this claim: {state['claim']}")
        ]
    
    # Get response
    response = llm_with_tools.invoke(messages)
    
    # Check if tool was called
    if response.tool_calls:
        return {
            "messages": messages + [response],
            "search_performed": True
        }
    else:
        # Parse final answer
        content = response.content
        
        # Simple parsing
        verdict = "BS" if "BS" in content.upper() else "LEGITIMATE"
        confidence = 85 if state.get('search_performed') else 70
        
        return {
            "messages": messages + [response],
            "verdict": verdict,
            "confidence": confidence,
            "reasoning": content
        }

## Build the Graph with Tools

In [26]:
# Create graph
graph_builder = StateGraph(State)

# Add nodes
graph_builder.add_node("detector", bs_detector)
graph_builder.add_node("tools", ToolNode(tools))

# Set entry
graph_builder.set_entry_point("detector")

# Add routing
def route_detector(state: State) -> str:
    """Route based on whether tool was called"""
    messages = state.get("messages", [])
    if messages and messages[-1].tool_calls:
        return "tools"
    else:
        return "end"

graph_builder.add_conditional_edges(
    "detector",
    route_detector,
    {"tools": "tools", "end": END}
)

# After tools, go back to detector
graph_builder.add_edge("tools", "detector")

# Compile
graph = graph_builder.compile()

## Test the Tool-Enhanced Detector

In [27]:
def check_with_tools(claim: str) -> dict:
    """Check claim with tool support"""
    result = graph.invoke({
        "claim": claim,
        "messages": [],
        "search_performed": False
    })
    return result

# Test claims
test_claims = [
    "The Boeing 747 has six engines",
    "The Wright brothers first flew in 1920",
    "Commercial planes fly at 50,000 feet"
]

for claim in test_claims:
    print(f"\n🔍 Checking: {claim}")
    result = check_with_tools(claim)
    
    print(f"📊 Verdict: {result['verdict']} ({result['confidence']}%)")
    print(f"🔧 Used search: {result.get('search_performed', False)}")
    print(f"💭 {result['reasoning'][:100]}...")


🔍 Checking: The Boeing 747 has six engines


BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[0].role', 'code': None}}

## Interactive Testing

In [ ]:
print("BS Detector with Search (type 'quit' to exit)\n")

while True:
    claim = input("Enter aviation claim: ")
    if claim.lower() == 'quit':
        break
    
    print("\nAnalyzing...")
    result = check_with_tools(claim)
    
    print(f"\n✅ {result['verdict']} ({result['confidence']}%)")
    if result.get('search_performed'):
        print("🔍 (Verified with search)")
    print(f"💭 {result['reasoning']}\n")

## What We Added

✅ **Tool Integration**: LLM can search for facts  
✅ **Higher Confidence**: Evidence-based verdicts  
✅ **Tool Node**: Automatic tool execution  
✅ **Message History**: Track conversation flow  

### How It Works

1. LLM receives claim
2. Decides if search is needed
3. Calls search tool if necessary
4. Uses results to make final verdict

**Next**: In Part 4, we'll add memory to remember past checks!